In [ ]:
import json
import h5py
import numpy as np
import json

from coco import COCO_CLASSES
from tqdm import tqdm

In [ ]:
# Set to root of the VisualGenome dataset
ROOT_VG = "/Users/ozzy/Projects/Datasets/VisualGenome/"
with open(ROOT_VG + 'objects.json') as json_file:
    objects_file = json.load(json_file)

In [ ]:
# Each annotated instance in Visual Genome has 'synsets' and 'names'
# Here we build a mapper which lists all names belonging to each synset in the Visual Genome dataset
SYNSET_TO_NAME_MAPPER, no_synset = {}, {}
for c, objects in tqdm(enumerate(objects_file), total=len(objects_file)):
    objects = objects['objects']
    for object_i in objects:
        
        if object_i['synsets'] == []:
            for noname in object_i['names']:
                if no_synset not in list(no_synset.keys()):
                    no_synset[noname] = 1
                else:
                    no_synset[noname] += 1
            continue
            
        stripped_names_i = [synset_i.split(".")[0] for synset_i in object_i["synsets"]]
        keys = list(SYNSET_TO_NAME_MAPPER.keys())
        
        for l in stripped_names_i:
            objects_temp = list(set(object_i["names"]))
            l = l.replace('_', ' ')
            if l not in keys:
                SYNSET_TO_NAME_MAPPER[l] = objects_temp
            else:
                if len(objects_temp) == 1:
                    if objects_temp[0] not in SYNSET_TO_NAME_MAPPER[l]:
                        SYNSET_TO_NAME_MAPPER[l].extend(objects_temp)
                else:
                    for o in objects_temp:
                        if o not in SYNSET_TO_NAME_MAPPER[l]:
                            SYNSET_TO_NAME_MAPPER[l].append(o)  


In [ ]:
# We build a mapper with each COCO class that appears in one of the names of the Visual Genome synsets
COCO_TO_VG_SYNSET_MAPPER = {}
for coco in tqdm(list(COCO_CLASSES.values())[1:]):
    for synset, names_list in list(SYNSET_TO_NAME_MAPPER.items()):
        if coco == "sports ball":
            if "ball" in names_list or "baseball" in names_list or "tennis ball" in names_list or "basketball" in names_list:
                synset = synset.replace("_", " ")
                if coco not in list(COCO_TO_VG_SYNSET_MAPPER.keys()):
                    COCO_TO_VG_SYNSET_MAPPER[coco] = [synset]              
                else:
                    COCO_TO_VG_SYNSET_MAPPER[coco].append(synset)
        if coco in names_list:
            synset = synset.replace("_", " ")
            if coco not in list(COCO_TO_VG_SYNSET_MAPPER.keys()):
                COCO_TO_VG_SYNSET_MAPPER[coco] = [synset]              
            else:
                COCO_TO_VG_SYNSET_MAPPER[coco].append(synset)


In [ ]:
# Count the number of occurrences of COCO classes in Visual Genome
COCO_CLASSES_IN_VG_COUNT = {}
for c, objects in tqdm(enumerate(objects_file), total=len(objects_file)):
    objects = objects['objects']
    for object_i in objects:
        if object_i['synsets'] == []:
            continue

        keys = list(COCO_CLASSES_IN_VG_COUNT.keys())
        stripped_names_i = [synset_i.split(".")[0] for synset_i in object_i["synsets"]]
        
        for l in stripped_names_i:
            l = l.replace('_', ' ')
            for k in COCO_TO_VG_SYNSET_MAPPER:
                key_list = COCO_TO_VG_SYNSET_MAPPER[k]
                if l in key_list:
                    if k not in keys:
                        COCO_CLASSES_IN_VG_COUNT[k] = 1
                    else:
                        COCO_CLASSES_IN_VG_COUNT[k] += 1
                else:
                    continue


In [ ]:
with open("SYNSET_TO_NAME_MAPPER.json", "w") as out1:
    json.dump(SYNSET_TO_NAME_MAPPER, out1)

with open("COCO_TO_VG_SYNSET_MAPPER.json", "w") as out2:
    json.dump(COCO_TO_VG_SYNSET_MAPPER, out2)

with open("COCO_CLASSES_IN_VG_COUNT.json", "w") as out3:
    json.dump(COCO_CLASSES_IN_VG_COUNT, out3)
    